In [ ]:
import requests

# Your OpenWeather API key
api_key = "596eadfb2fd863480d863b7062bd2e8f"

# Base URL for current weather data
base_url = "http://api.openweathermap.org/data/2.5/weather"

# Parameters: city and API key
params = {
    "q": "India",        # Change to your city of interest
    "appid": api_key,
    "units": "metric"     # Use "imperial" for Fahrenheit
}

# Make the GET request
response = requests.get(base_url, params=params)

# Check for successful response
if response.status_code == 200:
    data = response.json()
    print(f"Weather in {data['name']}:")
    print(f"Temperature: {data['main']['temp']}°C")
    print(f"Weather: {data['weather'][0]['description']}")
else:
    print(f"Error: {response.status_code} - {response.text}")


Weather in Innichen:
Temperature: 12.6°C
Weather: overcast clouds


In [2]:
pip install osmium

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 3.3 MB/s eta 0:00:00


In [ ]:
import osmium as osm
import pandas as pd
import requests
import time

# --- Step 1: Extract nodes ---
class NodeHandler(osm.SimpleHandler):
    def __init__(self):
        super(NodeHandler, self).__init__()
        self.nodes = []

    def node(self, n):
        self.nodes.append({
            "id": n.id,
            "lat": n.location.lat,
            "lon": n.location.lon,
            "tags": dict(n.tags)
        })

osm_file = "map.osm"
handler = NodeHandler()
handler.apply_file(osm_file)
df = pd.DataFrame(handler.nodes)

# --- Step 2: Reverse geocoding with Nominatim ---
def reverse_geocode(lat, lon):
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {
        "lat": lat,
        "lon": lon,
        "format": "json"
    }
    headers = {"User-Agent": "fleet-optimizer"}
    response = requests.get(url, params=params, headers=headers)

    if response.status_code == 200:
        return response.json().get("display_name", None)
    else:
        return None

# --- Step 3: Add location names (just first few to avoid rate limits) ---
df["location_name"] = df.head(15).apply(
    lambda row: reverse_geocode(row["lat"], row["lon"]), axis=1
)

print(df[["id","location_name"]])


                id                                      location_name
0         30037233  1126, EVR Periyar Salai, Periamet, Ward 58, Zo...
1         30037235  Langs Garden Park, Mahaveer Colony, Ward 58, Z...
2         30037236  EVK Sampath Salai, Mahaveer Colony, Ward 61, Z...
3         30037237  EVR Periyar Salai, Mahaveer Colony, Ward 61, Z...
4         30037238  EVR Periyar Salai, Mahaveer Colony, Ward 61, Z...
...            ...                                                ...
21100  13084021776                                                NaN
21101  13084021777                                                NaN
21102  13084021778                                                NaN
21103  13084021783                                                NaN
21104  13084021784                                                NaN

[21105 rows x 2 columns]


In [5]:
import random
import datetime
import osmium as osm
import pandas as pd
import requests

# --- Step 1: Simulated traffic function ---
def get_simulated_traffic(zone, timestamp):
    hour = timestamp.hour
    if zone == "Downtown":
        base_speed = 20
    elif zone == "Airport":
        base_speed = 40
    else:
        base_speed = 50

    # Congestion effect (rush hours)
    if 7 <= hour <= 9 or 17 <= hour <= 19:
        speed = base_speed * random.uniform(0.5, 0.8)
    else:
        speed = base_speed * random.uniform(0.9, 1.2)

    return round(speed, 2)

# --- Step 2: Extract OSM nodes ---
class NodeHandler(osm.SimpleHandler):
    def __init__(self):
        super(NodeHandler, self).__init__()
        self.nodes = []

    def node(self, n):
        self.nodes.append({
            "id": n.id,
            "lat": n.location.lat,
            "lon": n.location.lon,
            "tags": dict(n.tags)
        })

osm_file = "map.osm"
handler = NodeHandler()
handler.apply_file(osm_file)
df = pd.DataFrame(handler.nodes)

# --- Step 3: Reverse geocoding (sample only, due to rate limits) ---
def reverse_geocode(lat, lon):
    url = "https://nominatim.openstreetmap.org/reverse"
    params = {"lat": lat, "lon": lon, "format": "json"}
    headers = {"User-Agent": "fleet-optimizer"}
    response = requests.get(url, params=params, headers=headers)
    if response.status_code == 200:
        return response.json().get("display_name", None)
    return None

df["location_name"] = df.head(10).apply(
    lambda row: reverse_geocode(row["lat"], row["lon"]), axis=1
)

# --- Step 4: Assign traffic speed based on detected zone ---
def detect_zone(location_name):
    if location_name is None or not isinstance(location_name, str):
        return "Other"
    name = location_name.lower()
    if "downtown" in name:
        return "Downtown"
    elif "airport" in name:
        return "Airport"
    elif "suburb" in name:
        return "Suburb"
    else:
        return "Other"

# Apply zone detection
df["zone"] = df["location_name"].apply(detect_zone)

# Simulate traffic for each node (using current time)
current_time = datetime.datetime.now()
df["traffic_speed_kmh"] = df["zone"].apply(
    lambda z: get_simulated_traffic(z, current_time)
)

print(df[["id", "location_name", "zone", "traffic_speed_kmh"]])

                id                                      location_name   zone  \
0         30037233  1126, EVR Periyar Salai, Periamet, Ward 58, Zo...  Other   
1         30037235  Langs Garden Park, Mahaveer Colony, Ward 58, Z...  Other   
2         30037236  EVR Periyar Salai, Mahaveer Colony, Ward 61, Z...  Other   
3         30037237  EVR Periyar Salai, Mahaveer Colony, Ward 61, Z...  Other   
4         30037238  EVR Periyar Salai, Mahaveer Colony, Ward 61, Z...  Other   
...            ...                                                ...    ...   
21100  13084021776                                                NaN  Other   
21101  13084021777                                                NaN  Other   
21102  13084021778                                                NaN  Other   
21103  13084021783                                                NaN  Other   
21104  13084021784                                                NaN  Other   

       traffic_speed_kmh  
0           

In [7]:
import random
import pandas as pd
import datetime
import numpy as np

# Load fleet dataset
fleet_df = pd.read_csv("/content/synthetic_fleet_basic.csv")

# Map pickup locations to lat/lon
location_coords = {
    "Yeshwanthpur": (13.0186, 77.5560),
    "Whitefield": (12.9698, 77.7499),
    "Koramangala": (12.9352, 77.6245),
    "Marathahalli": (12.9569, 77.7011),
    "Indiranagar": (12.9784, 77.6408),
    "Electronic City": (12.8452, 77.6602)
}

# Define average demand intensity (requests/hour) for each zone
zone_intensity = {
    "Yeshwanthpur": 8,
    "Whitefield": 15,
    "Koramangala": 12,
    "Marathahalli": 10,
    "Indiranagar": 7,
    "Electronic City": 6
}

# Simulate demand for 24 hours
demand_records = []
start_time = datetime.datetime(2025, 8, 28, 0, 0)

for hour in range(24):
    for zone, (lat, lon) in location_coords.items():
        intensity = zone_intensity.get(zone, 5)
        num_requests = np.random.poisson(intensity)

        for _ in range(num_requests):
            minute = random.randint(0, 59)
            demand_time = start_time + datetime.timedelta(hours=hour, minutes=minute)

            # Demand size (scaled based on area intensity, 50–200 kg typical)
            demand_size = random.randint(50, min(200, int(intensity * 20)))

            demand_records.append({
                "time": demand_time,
                "zone": zone,
                "lat": lat,
                "lon": lon,
                "demand_size_kg": demand_size
            })

# Convert to DataFrame
demand_df = pd.DataFrame(demand_records)
print(demand_df.head())


                 time          zone      lat     lon  demand_size_kg
0 2025-08-28 00:01:00  Yeshwanthpur  13.0186  77.556             105
1 2025-08-28 00:53:00  Yeshwanthpur  13.0186  77.556              72
2 2025-08-28 00:38:00  Yeshwanthpur  13.0186  77.556              70
3 2025-08-28 00:03:00  Yeshwanthpur  13.0186  77.556              81
4 2025-08-28 00:33:00  Yeshwanthpur  13.0186  77.556              57


In [9]:
import random
import datetime
import numpy as np
import pandas as pd
import osmium as osm
import requests

# ============================================================
# 1. DEMAND DATA
# ============================================================

location_coords = {
    "Yeshwanthpur": (13.0186, 77.5560),
    "Whitefield": (12.9698, 77.7499),
    "Koramangala": (12.9352, 77.6245),
    "Marathahalli": (12.9569, 77.7011),
    "Indiranagar": (12.9784, 77.6408),
    "Electronic City": (12.8452, 77.6602)
}

zone_intensity = {
    "Yeshwanthpur": 8,
    "Whitefield": 15,
    "Koramangala": 12,
    "Marathahalli": 10,
    "Indiranagar": 7,
    "Electronic City": 6
}

demand_records = []
start_time = datetime.datetime(2025, 8, 28, 0, 0)

for hour in range(6):  # shorter simulation for demo
    for zone, (lat, lon) in location_coords.items():
        intensity = zone_intensity.get(zone, 5)
        num_requests = np.random.poisson(intensity)
        for _ in range(num_requests):
            minute = random.randint(0, 59)
            demand_time = start_time + datetime.timedelta(hours=hour, minutes=minute)
            demand_size = random.randint(50, min(200, int(intensity * 20)))
            demand_records.append({
                "demand_id": f"d_{hour}_{random.randint(1000,9999)}",
                "time": demand_time,
                "zone": zone,
                "lat": lat,
                "lon": lon,
                "demand_size_kg": demand_size
            })

demand_df = pd.DataFrame(demand_records)
print("Demand sample:\n", demand_df.head())

# ============================================================
# 2. SIMPLE FLEET DATA
# ============================================================

fleet_df = pd.DataFrame([
    {"vehicle_id": "V1", "capacity": 500, "lat": 12.9716, "lon": 77.5946, "availability": 1},
    {"vehicle_id": "V2", "capacity": 300, "lat": 12.9352, "lon": 77.6245, "availability": 1},
    {"vehicle_id": "V3", "capacity": 700, "lat": 13.0186, "lon": 77.5560, "availability": 0}
])
print("\nFleet sample:\n", fleet_df.head())

# ============================================================
# 3. TRAFFIC DATA (SIMULATED SPEEDS)
# ============================================================

def get_simulated_traffic(zone, timestamp):
    base_speed = 40 if zone == "Airport" else 30
    if 7 <= timestamp.hour <= 9 or 17 <= timestamp.hour <= 19:
        return base_speed * random.uniform(0.5, 0.8)  # rush hour slowdown
    return base_speed * random.uniform(0.9, 1.2)

demand_df["traffic_speed_kmh"] = demand_df.apply(
    lambda r: get_simulated_traffic(r["zone"], r["time"]), axis=1
)

# ============================================================
# 4. WEATHER DATA
# ============================================================

# Dummy API call (replace with OpenWeather)
weather_data = {"weather": [{"description": "clear"}]}
weather_factor = 1.2 if "rain" in weather_data["weather"][0]["description"].lower() else 1.0

# ============================================================
# 5. STATE VECTOR CREATION
# ============================================================

state_vectors = []

for _, vehicle in fleet_df.iterrows():
    for _, demand in demand_df.iterrows():
        distance = np.linalg.norm([vehicle.lat - demand.lat, vehicle.lon - demand.lon])
        eta = distance / (demand.traffic_speed_kmh / 60 + 1e-6)  # minutes approx
        state = [
            vehicle.capacity,
            vehicle.availability,
            demand.demand_size_kg,
            demand.traffic_speed_kmh * weather_factor,  # adjusted traffic
            eta,
            distance
        ]
        state_vectors.append(state)

state_matrix = np.array(state_vectors)

print("\nState Matrix Shape:", state_matrix.shape)
print("Sample State Vector (first row):\n", state_matrix[0])


Demand sample:
   demand_id                time          zone      lat     lon  demand_size_kg
0  d_0_9637 2025-08-28 00:35:00  Yeshwanthpur  13.0186  77.556             143
1  d_0_6603 2025-08-28 00:13:00  Yeshwanthpur  13.0186  77.556             135
2  d_0_3755 2025-08-28 00:04:00  Yeshwanthpur  13.0186  77.556             104
3  d_0_2648 2025-08-28 00:09:00  Yeshwanthpur  13.0186  77.556             156
4  d_0_6364 2025-08-28 00:03:00  Yeshwanthpur  13.0186  77.556              92

Fleet sample:
   vehicle_id  capacity      lat      lon  availability
0         V1       500  12.9716  77.5946             1
1         V2       300  12.9352  77.6245             1
2         V3       700  13.0186  77.5560             0

State Matrix Shape: (1014, 6)
Sample State Vector (first row):
 [5.00000000e+02 1.00000000e+00 1.43000000e+02 3.02837790e+01
 1.20498083e-01 6.08190760e-02]
